In [1]:
from dotenv import load_dotenv , find_dotenv

_ = load_dotenv(find_dotenv())

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

openai_embedding = OpenAIEmbeddings()
vectordb = Chroma(embedding_function=openai_embedding,persist_directory="db/chroma/")

In [4]:
# basic similarity search
question = "What are major subjects for this class?"
similar_docs = vectordb.similarity_search(question,k = 4)

In [5]:
# language model
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.0)

llm.predict("hello")

'Hello! How can I assist you today?'

In [16]:
# build a prompt using template 

from langchain.prompts import PromptTemplate
qa_template = """
Use the context delimited by ``` to answer the question at the end.
Note: If you cannot answer the question, respond with "Not enough information in Provided Context", do not make up an answer on your own.
Context : 
```{context}```
Question : {question}
Answer is : 
"""

QA_PROMPT = PromptTemplate.from_template(qa_template)

In [22]:
from langchain.chains import RetrievalQA

question = "Is probability a class topic ?"

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectordb.as_retriever(),
    return_source_documents = True,
    chain_type_kwargs={"prompt":QA_PROMPT}
)

qa_chain


RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='\nUse the context delimited by ``` to answer the question at the end.\nNote: If you cannot answer the question, respond with "Not enough information in Provided Context", do not make up an answer on your own.\nContext : \n```{context}```\nQuestion : {question}\nAnswer is : \n', template_format='f-string', validate_template=True), llm=ChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.c

In [19]:
import langchain
langchain.debug = False
result = qa_chain({"query":question})

In [19]:
print(result['result'])

Yes, probability is a class topic.


Add memory to qa_retrieval

In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [25]:
import langchain
langchain.debug = True
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()
qa_chat = ConversationalRetrievalChain.from_llm( # takes history and new question and condenses it into a new qustion , to pass to vector store
    llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt" : QA_PROMPT}
)

qa_chat

ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Is probability a class topic?', additional_kwargs={}, example=False), AIMessage(content='Yes, probability is a topic that will be covered in this class. The instructor assumes familiarity with basic probability and statistics, so it is expected that students have prior knowledge in this area.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['

In [26]:
#langchain.debug = False
question = "Is probability a class topic?"
result = qa_chat({"question": question})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Is probability a class topic?",
  "chat_history": "\nHuman: Is probability a class topic?\nAssistant: Yes, probability is a topic that will be covered in this class. The instructor assumes familiarity with basic probability and statistics, so it is expected that students have prior knowledge in this area."
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: Is probability a class topic?\nAssistant: Yes, probability is a topic that will be covered in this class. The instructor assumes familiarity with basic proba

In [28]:
print(result['answer'])
print(memory)

Yes, probability is covered in this class.
chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Is probability a class topic?', additional_kwargs={}, example=False), AIMessage(content='Yes, probability is a topic that will be covered in this class. The instructor assumes familiarity with basic probability and statistics, so it is expected that students have prior knowledge in this area.', additional_kwargs={}, example=False), HumanMessage(content='Is probability a class topic?', additional_kwargs={}, example=False), AIMessage(content='Yes, probability is covered in this class.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=True human_prefix='Human' ai_prefix='AI' memory_key='chat_history'


In [28]:
question = "Why are those prerequisites needed?"
result = qa_chat({"question":question})

In [29]:
print(result['answer'])

The instructor assumes familiarity with basic probability and statistics because these concepts are fundamental to understanding and applying machine learning algorithms. Probability and statistics provide the foundation for understanding uncertainty, making predictions, and evaluating the performance of machine learning models. Without a basic understanding of these concepts, it would be challenging to grasp the underlying principles and techniques used in machine learning.


In [ ]:
llm.predict_messages()